In [1]:
import time

import sounddevice as sd
import wave
import soundcard as sc
import numpy
import pyaudio

from pathlib import Path
from concurrent.futures import ThreadPoolExecutor

from distributed import Client
from scipy.io import wavfile

wav_in_path = Path('../media/clip_6.wav')
fs_wav, wav_arr = wavfile.read(wav_in_path)

executor = ThreadPoolExecutor()

# Soundcard

In [2]:
# get a list of all speakers:
speakers = sc.all_speakers()
print("Speakers:")
for s in speakers:
    print(s)
# get the current default speaker on your system:
default_speaker = sc.default_speaker()
print(f"\nDefault speaker is: {default_speaker}\n")

# get a list of all microphones:
mics = sc.all_microphones()
print("Microphones:")
for m in mics:
    print(m)
# get the current default microphone on your system:
default_mic = sc.default_microphone()
print(f"\nDefault mic is: {default_mic}\n")

# search for a sound card by substring:
headphone_speaker = sc.get_speaker('Mono')
headphone_mic = sc.get_microphone('Mono')

komplete_input = sc.get_microphone('Komplete')
komplete_output = sc.get_speaker('Komplete')

Speakers:
<Speaker Built-in Audio Analog Stereo (2 channels)>
<Speaker SteelSeries Arctis 7 Analog Mono (1 channels)>
<Speaker SteelSeries Arctis 7 Analog Stereo (2 channels)>
<Speaker Komplete Audio 6 Analog Surround 2.1 (3 channels)>

Default speaker is: <Speaker SteelSeries Arctis 7 Analog Stereo (2 channels)>

Microphones:
<Microphone Built-in Audio Analog Stereo (2 channels)>
<Microphone SteelSeries Arctis 7 Analog Mono (1 channels)>
<Microphone Komplete Audio 6 Multichannel (6 channels)>

Default mic is: <Microphone Komplete Audio 6 Multichannel (6 channels)>



In [3]:
komplete_input

<Microphone Komplete Audio 6 Multichannel (6 channels)>

In [4]:
komplete_output

<Speaker Komplete Audio 6 Analog Surround 2.1 (3 channels)>

In [5]:
with komplete_input.recorder(samplerate=44100, channels=[4,5]) as input_, \
     komplete_output.player(samplerate=44100, channels=[0,1]) as output_:
    executor.submit(output_.play(wav_arr))
    data = executor.submit(input_.record, numframes=wav_arr.shape[0])

mod_wav_arr = data.result()

In [6]:
mod_wav_arr.max()

0.00022334074

In [ ]:
with headphone_speaker.player(samplerate=44_100, channels=2) as sp:
    sp.play(mod_wav_arr*3000)

In [20]:
mod_wav_arr = data.result()

In [21]:
mod_wav_arr.max()

7.461679e-05

In [ ]:
wav_arr

In [ ]:
mod_wav_arr.max()

In [ ]:
wav_arr

In [ ]:
data.shape

In [ ]:
one_mic

In [ ]:
one_speaker

In [ ]:
speakers[1]

In [ ]:


# alternatively, get a `Recorder` and `Player` object
# and play or record continuously:
#with mics[1].recorder(samplerate=48000) as mic, \
#      speakers[1].player(samplerate=48000) as sp:
#    for _ in range(100):
#        data = mic.record(numframes=1024)
#        sp.play(data)

In [ ]:
#data = data * 20

In [ ]:
with speakers[1].player(samplerate=48_000) as sp:
    sp.play(data)

# pyaudio

3 in on soundcard -> L out (left output blackbox)
4 in on soundcard -> R out (left output blackbox)
3 out on soundcard -> L in (left output blackbox)
4 out on soundcard -> R in (right input blackbox - MONO)


wav file goes out outputs of soundcard (in DAW) -> cables -> input of black box -> modification -> output of black box -> input of sound card

In [7]:
p = pyaudio.PyAudio()
info = p.get_host_api_info_by_index(0)
numdevices = info.get('deviceCount')
for i in range(0, numdevices):
        if (p.get_device_info_by_host_api_device_index(0, i).get('maxInputChannels')) > 0:
            print("Input Device id ", i, " - ", p.get_device_info_by_host_api_device_index(0, i).get('name'))

Input Device id  6  -  SteelSeries Arctis 7: USB Audio (hw:1,0)


In [ ]:
"""
PyAudio Example: Make a wire between input and output (i.e., record a
few samples and play them back immediately).

This is the callback (non-blocking) version.
"""

WIDTH = 2
CHANNELS = wav_arr.shape[0]
RATE = fs_wav

p = pyaudio.PyAudio()

In [ ]:
p

In [ ]:
def callback(in_data, frame_count, time_info, status):
    return (in_data, pyaudio.paContinue)

stream = p.open(format=p.get_format_from_width(WIDTH),
                channels=CHANNELS,
                rate=RATE,
                input=True,
                output=True,
                stream_callback=callback)

In [ ]:
stream.start_stream()

while stream.is_active():
    time.sleep(0.1)

stream.stop_stream()
stream.close()

p.terminate()

In [ ]:
CHUNK = 1024 * 3  # Chunk is the bytes which are currently processed
FORMAT = pyaudio.paInt16
RATE = 44100
RECORD_SECONDS = 2
WAVE_OUTPUT = str(wav_in_path)
p = pyaudio.PyAudio()
stream = p.open(format=FORMAT,channels=6,rate=RATE,input=True,frames_per_buffer=CHUNK,input_device_index=4)

frames = []  # np array storing all the data

for i in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
    data = stream1.read(CHUNK)
    frames.append(data1)




stream.stop_stream()
stream.close()
p.terminate()

wavef = wave.open(WAVE_OUTPUT, 'wb')  # opening the file
wavef.setnchannels(1)
wavef.setsampwidth(p.get_sample_size(FORMAT))
wavef.setframerate(RATE)
wavef.writeframes(b''.join(frames1))  # writing the data to be saved
wavef.close()

# Python SoundDevice

In [ ]:
#!/usr/bin/env python3
"""Pass input directly to output.

https://app.assembla.com/spaces/portaudio/git/source/master/test/patest_wire.c

"""
print(sd.query_devices())

In [ ]:

input_device = 8# numeric id or substring)
output_device = 8# same
channels = 2 # no. channels
dtype = None# audio data type
samplerate = 44100. #
blocksize = 1024 #??
latency = 0.001 #??


In [ ]:
# steps:
# 1. play wav file through soundcard output
# 2. listen to and record soundcard input

In [ ]:

def callback(indata, outdata, frames, time, status):
    if status:
        print(status)
    outdata[:] = indata


try:
    with sd.Stream(device=(input_device, output_device),
                   samplerate=samplerate, blocksize=blocksize,
                   dtype=dtype, latency=latency,
                   channels=channels, callback=callback):
        print('#' * 80)
        print('press Return to quit')
        print('#' * 80)
        input()
except KeyboardInterrupt:
    parser.exit('')
except Exception as e:
    parser.exit(type(e).__name__ + ': ' + str(e))

# PYO?